In [1]:
import pandas as pd

# Load your DataFrame (assuming it's saved as 'data.csv')
df = pd.read_csv('train_with_features.csv')


In [2]:
df.shape

(223, 15)

In [3]:
df.head()

,Unnamed: 0,spotify_track_id,song_title,label,emotion,session_type,subject_id,id,song_author,alpha_power,beta_power,delta_power,theta_power,variance,mean
0,0,06s6aloy62vytl3MnT6gfl,Times Like These - Live at the Pantages Theatr...,0,Amusement,personal,23,3784258358,['Foo Fighters'],2.258961e-12,1.019803e-12,4.672743e-11,4.411881e-12,2.553564e-10,2.346306e-09
1,1,0h9fnCSnbUgOEgibnQByFv,Everyday Life,2,Disappointment,personal,23,2395445698,['Coldplay'],2.269191e-12,7.160127e-13,2.629792e-11,3.541972e-12,2.081659e-10,1.011783e-08
2,2,1iArQTuOzxvrtniGmkyy92,Burden In My Hand,3,Anger,personal,23,3049220457,['Soundgarden'],3.825635e-12,1.182833e-12,6.786013e-11,5.654127e-12,5.708281e-10,-1.731397e-07
3,3,3A9vIxzGBjEfqmDK7H9exS,Pyramid Song,3,Fear,personal,23,2667905835,['Radiohead'],3.852573e-12,1.220438e-12,5.783148e-11,6.408991e-12,3.996517e-10,1.924873e-07
4,4,39kHMfF3dBMZMbOtoit1XF,On The Mend,0,Joy,personal,23,1281311748,['Foo Fighters'],3.885899e-12,1.389423e-12,8.960010e-11,7.100184e-12,4.647378e-10,6.473157e-08


In [4]:

# Define your features and target variable
X = df[['alpha_power','beta_power','delta_power','theta_power']]
y = df['label']
X.head()


,alpha_power,beta_power,delta_power,theta_power
0,2.258961e-12,1.019803e-12,4.672743e-11,4.411881e-12
1,2.269191e-12,7.160127e-13,2.629792e-11,3.541972e-12
2,3.825635e-12,1.182833e-12,6.786013e-11,5.654127e-12
3,3.852573e-12,1.220438e-12,5.783148e-11,6.408991e-12
4,3.885899e-12,1.389423e-12,8.960010e-11,7.100184e-12


In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [6]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [13]:
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Initialize the model
model = XGBClassifier( eval_metric='mlogloss')
# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

In [14]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.3111111111111111
Classification Report:
               precision    recall  f1-score   support

           0       0.31      1.00      0.47        14
           1       0.00      0.00      0.00        13
           2       0.00      0.00      0.00         9
           3       0.00      0.00      0.00         9

    accuracy                           0.31        45
   macro avg       0.08      0.25      0.12        45
weighted avg       0.10      0.31      0.15        45



/Users/harsh_dadwal/Harsh/Research_signal/A.Purohit/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/harsh_dadwal/Harsh/Research_signal/A.Purohit/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/harsh_dadwal/Harsh/Research_signal/A.Purohit/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to cont

In [15]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2]
}

grid_search = GridSearchCV(model, param_grid, cv=5)
grid_search.fit(X_train, y_train)

print("Best parameters:", grid_search.best_params_)

Best parameters: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100}


In [16]:
best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train)

# Evaluate the final model
final_predictions = best_model.predict(X_test)
print("Final Model Accuracy:", accuracy_score(y_test, final_predictions))

Final Model Accuracy: 0.3111111111111111


In [17]:
df['label'].value_counts()

label
0    68
1    65
3    47
2    43
Name: count, dtype: int64